# Testing and Assertions

In this lesson we are going to cover defensive programming and testing, two techniques that improve the quality of the programs we write by lowering the rate of bugs in our code and forcing us to think about program design and purpose in advance. We are going to start with learning about using assertions.

## Assertions
The first step toward getting the right answers from our programs is to assume that mistakes will happen and to guard against them. This is called defensive programming, and the most common way to do it is to add assertions to our code so that it checks itself as it runs. An assertion is simply a statement that something must be true at a certain point in a program. When Python sees one, it evaluates the assertion’s condition. If it’s true, Python does nothing, but if it’s false, Python halts the program immediately and prints the error message if one is provided. For example, this piece of code halts as soon as the loop encounters a value that isn’t positive:



In [6]:

numbers = [1.5, 2.3, 0.7, -0.001, 4.4]
total = 0.0
for n in numbers:
    assert n > 0.0, 'Data should only contain positive values'
    total += n
print('total is:', total)


AssertionError: Data should only contain positive values

Programs like the Firefox browser are full of assertions: 10-20% of the code they contain are there to check that the other 80-90% are working correctly. Broadly speaking, assertions fall into three categories:

* A precondition is something that must be true at the start of a function in order for it to work correctly.

* A postcondition is something that the function guarantees is true when it finishes.

* An invariant is something that is always true at a particular point inside a piece of code.

For example, suppose we are representing rectangles using a tuple of four coordinates `(x0, y0, x1, y1)`, representing the lower left and upper right corners of the rectangle. In order to do some calculations, we need to normalize the rectangle so that the lower left corner is at the origin and the longest side is 1.0 units long. This function does that, but checks that its input is correctly formatted and that its result makes sense:



In [1]:
def normalize_rectangle(rect):
    '''Normalizes a rectangle so that it is at the origin and 1.0 units long on its longest axis.'''
    assert len(rect) == 4, 'Rectangles must contain 4 coordinates'
    x0, y0, x1, y1 = rect
    assert x0 < x1, 'Invalid X coordinates'
    assert y0 < y1, 'Invalid Y coordinates'

    dx = x1 - x0
    dy = y1 - y0
    if dx > dy:
        scaled = float(dx) / dy
        upper_x, upper_y = 1.0, scaled
    else:
        scaled = float(dx) / dy
        upper_x, upper_y = scaled, 1.0

    assert 0 < upper_x <= 1.0, 'Calculated upper X coordinate invalid'
    assert 0 < upper_y <= 1.0, 'Calculated upper Y coordinate invalid'

    return (0, 0, upper_x, upper_y)


The preconditions on lines 2, 4, and 5 catch invalid inputs:





The post-conditions help us catch bugs by telling us when our calculations cannot have been correct. For example, if we normalize a rectangle that is taller than it is wide everything seems OK:


In [2]:
print(normalize_rectangle( (0.0, 0.0, 1.0, 5.0) ))


(0, 0, 0.2, 1.0)


but if we normalize one that’s wider than it is tall, the assertion is triggered:


In [3]:
print(normalize_rectangle( (0.0, 0.0, 5.0, 1.0) ))

AssertionError: Calculated upper Y coordinate invalid

Re-reading our function, we realize that line 10 should divide `dy` by `dx` rather than `dx` by `dy`. (You can display line numbers by typing Ctrl-M, then L.) If we had left out the assertion at the end of the function, we would have created and returned something that had the right shape as a valid answer, but wasn’t. Detecting and debugging that would almost certainly have taken more time in the long run than writing the assertion.

But assertions aren’t just about catching errors: they also help people understand programs. Each assertion gives the person reading the program a chance to check (consciously or otherwise) that their understanding matches what the code is doing.

Most good programmers follow two rules when adding assertions to their code. The first is, fail early, fail often. The greater the distance between when and where an error occurs and when it’s noticed, the harder the error will be to debug, so good code catches mistakes as early as possible.

The second rule is, turn bugs into assertions or tests. Whenever you fix a bug, write an assertion that catches the mistake should you make it again. If you made a mistake in a piece of code, the odds are good that you have made other mistakes nearby, or will make the same mistake (or a related one) the next time you change it. Writing assertions to check that you haven’t regressed (i.e., haven’t re-introduced an old problem) can save a lot of time in the long run, and helps to warn people who are reading the code (including your future self) that this bit is tricky.

# Test-Driven Development

An assertion checks that something is true at a particular point in the program. The next step is to check the overall behavior of a piece of code, i.e., to make sure that it produces the right output when it’s given a particular input. For example, suppose we need to find where two or more time series overlap. The range of each time series is represented as a pair of numbers, which are the time the interval started and ended. The output is the largest range that they all include:

<img src="http://swcarpentry.github.io/python-novice-inflammation/fig/python-overlapping-ranges.svg",width=500,height=500>

Most novice programmers would solve this problem like this:

1. Write a function `range_overlap`.
2. Call it interactively on two or three different inputs.
3. If it produces the wrong answer, fix the function and re-run that test.

This clearly works — after all, thousands of scientists are doing it right now — but there’s a better way:

1. Write a short function for each test.
2. Write a `range_overlap` function that should pass those tests.
3. If `range_overlap` produces any wrong answers, fix it and re-run the test functions.

Writing the tests before writing the function they exercise is called test-driven development (TDD). Its advocates believe it produces better code faster because:

1. If people write tests after writing the thing to be tested, they are subject to confirmation bias, i.e., they subconsciously write tests to show that their code is correct, rather than to find errors.
2. Writing tests helps programmers figure out what the function is actually supposed to do.

Here are three test functions for `range_overlap`:



In [6]:
assert range_overlap([ (0.0, 1.0) ]) == (0.0, 1.0)
assert range_overlap([ (2.0, 3.0), (2.0, 4.0) ]) == (2.0, 3.0)
assert range_overlap([ (0.0, 1.0), (0.0, 2.0), (-1.0, 1.0) ]) == (0.0, 1.0)

NameError: name 'range_overlap' is not defined

The error is actually reassuring: we haven’t written `range_overlap` yet, so if the tests passed, it would be a sign that someone else had and that we were accidentally using their function.

And as a bonus of writing these tests, we’ve implicitly defined what our input and output look like: we expect a list of pairs as input, and produce a single pair as output.

Something important is missing, though. We don’t have any tests for the case where the ranges don’t overlap at all:



In [7]:
assert range_overlap([ (0.0, 1.0), (5.0, 6.0) ]) == ???


SyntaxError: invalid syntax (<ipython-input-7-cf12f48f208b>, line 1)

What should `range_overlap` do in this case: fail with an error message, produce a special value like (0.0, 0.0) to signal that there’s no overlap, or something else? Any actual implementation of the function will do one of these things; writing the tests first helps us figure out which is best before we’re emotionally invested in whatever we happened to write before we realized there was an issue.

And what about this case?

In [8]:
assert range_overlap([ (0.0, 1.0), (1.0, 2.0) ]) == ???


SyntaxError: invalid syntax (<ipython-input-8-79ff80dec5ad>, line 1)

Do two segments that touch at their endpoints overlap or not? Mathematicians usually say “yes”, but engineers usually say “no”. The best answer is “whatever is most useful in the rest of our program”, but again, any actual implementation of `range_overlap` is going to do something, and whatever it is ought to be consistent with what it does when there’s no overlap at all.

Since we’re planning to use the range this function returns as the X axis in a time series chart, we decide that:

1. Every overlap has to have non-zero width, and
2. We will return the special value None when there’s no overlap.

`None` is built into Python, and means “nothing here”. (Other languages often call the equivalent value `null` or `nil`). With that decision made, we can finish writing our last two tests:

In [9]:
assert range_overlap([ (0.0, 1.0), (5.0, 6.0) ]) == None
assert range_overlap([ (0.0, 1.0), (1.0, 2.0) ]) == None


NameError: name 'range_overlap' is not defined

Again, we get an error because we haven’t written our function, but we’re now ready to do so:



In [11]:
def range_overlap(ranges):
    '''Return common overlap among a set of [low, high] ranges.'''
    lowest = 0.0
    highest = 1.0
    for (low, high) in ranges:
        lowest = max(lowest, low)
        highest = min(highest, high)
    return (lowest, highest)

(Take a moment to think about why we use `max` to raise `lowest` and `min` to lower `highest`). We’d now like to re-run our tests, but they’re scattered across three different cells. To make running them easier, let’s put them all in a function:



In [12]:
def test_range_overlap():
    assert range_overlap([ (0.0, 1.0), (5.0, 6.0) ]) == None
    assert range_overlap([ (0.0, 1.0), (1.0, 2.0) ]) == None
    assert range_overlap([ (0.0, 1.0) ]) == (0.0, 1.0)
    assert range_overlap([ (2.0, 3.0), (2.0, 4.0) ]) == (2.0, 3.0)
    assert range_overlap([ (0.0, 1.0), (0.0, 2.0), (-1.0, 1.0) ]) == (0.0, 1.0)

We can now test `range_overlap` with a single function call:



In [13]:
test_range_overlap()


AssertionError: 

The first test that was supposed to produce `None` fails, so we know something is wrong with our function. We don’t know whether the other tests passed or failed because Python halted the program as soon as it spotted the first error. Still, some information is better than none, and if we trace the behavior of the function with that input, we realize that we’re initializing `lowest` and `highest` to 0.0 and 1.0 respectively, regardless of the input values. This violates another important rule of programming: _always initialize from data_.





# Testing and Test Driven Development

It may seem as the the point of test driven development is to create software with fewer bugs thanks to testing. This is in part true. Perhaps more importantly TDD encourages you think about the purpose of the code  you are writing upfront. A good analogy TDD is like building a lock and key simultaneously to ensure they both fit. 

Testing can seem like wasted effort, why test when you can forge on and create more features? After all your code is working. For a lot of people new to programming the value of testing is only shown when they get to a stage where their code isn't working and they can't fix it, a situation which could have avoided by testing. Creating programs without testing feels fast and moves slowly, whilst using tests is the opposite.

You may be wondering at this point, when should I be creating and running tests for my programs? In short, the answer is we should always be testing. Always test unless the function is trivial.

There are a range of different sorts of tests used by software developers. Tests exist for every part of the software development cycle. The tests that you will likely find useful when creating scientific software are:
* Unit tests. These are written to test functions or areas of code, i.e units. They are written to ensure that functions are working as we expected. In the `range_overlap` the tests created are unit tests.
* Regression tests. These tests measure whether program output or behaviour are consistent with earlier versions of the program. 
* Integration tests. Integration tests are written  to test that the major parts of of a program work together as expected. 

There are other types of software tests, but these three are the most likely you will use and encounter if writing scientific software. For this module we are have been focusing on unit tests. As you become a more experienced programmer and build more complex pieces of software you might start to use other sorts of tests.



# The TDD cycle

The cycle of writing software is often summed up by Red, Green, Refactor.

* **_Red_: Write a test that doesn't run** 
    Think about what you want your code to accomplish and write a test as though the code already exists.
    
* **_Green_: Make the test work quickly** 
    Write enough code to make the test past. A common refrain at this stage is 'YAGNI' (You Ain't Gonna Need It'). Don't and generalize to quickly. Once all your tests are passed you are finished with this stage.
    
*  **_Refactor_: Make it right** 
    Clean up your code and remove unecessary duplication while ensuring that you continue to pass all tests. When we refactor we are improving the quality of our code without changing its external behaviour. From outside the code you pass in the same arguments and recieve the same results.



<img src="TDD_cycle.png",width=500,height=500>

When it comes to getting the test to pass quickly, there are techniques frequently used
* **Faking it**: Return constant values and gradually replace them until you have real code
* **Obvious Implementation**. Implement the obvious solution to the problem. Don't fall into the trap of trying to implement the "most" obvious solution, don't overthink it.
* **Triangulation**: Sometimes you want to generalize some code but are not sure how to do it. Triangulation is the process of adding test cases with different input and expected output to try and force us to genearlize our code to handle all the test cases.

Now for another example, to make this more concrete.

## Palindromes 

A classic basic programming task is to check whether a string is a palindrome. Though this is a fairly simple problem it will get you help you get in the swing of thinking about what you want your code to achieve before you start implementing it.

### Getting started

The most simple palindrome we could have is a string containing a single character. So lets start by writing a test for that case. 

In [5]:
def test_is_palindrome():
    assert is_palindrome("a") == True



Now that we have written our first test case, lets write the simplest code we can that will pass our test case. Keep in mind that we are writing just enough to pass our test case, don't try and prematurely cover tests we haven't written. Keep it simple and write code to pass the test. Have a go at completing the `is_palindrome` function and then compare your function to the one in the cell below (hit the **+** icon in the top left hand corner of the cell to unhide the cell with the answer)

In [6]:
## Write your code in here
def is_palindrome(pal):
    ???
test_is_palindrome()

Object `` not found.


AssertionError: 

In [7]:
def is_palindrome(pal):
    return len(pal) == 1

test_is_palindrome()

Our code has passed our test. Amazing. So lets add another test case, which is a bit more complex, a string with a length greater than 1. We have "realized" that we need to the ability to test whether a string longer than one character is a palindrome or not, so we have added a test to cover this new requirement.   We are pretty certain our code will won't pass this new test. 

In [1]:
def test_is_palindrome():
    assert is_palindrome("a") == True
    assert is_palindrome("bb" )== True
    
test_is_palindrome()

NameError: global name 'is_palindrome' is not defined

With the addition of the new test case, our code is failing to pass all our tests. So once again we are going to change our program to handle the new test case. When attempting this section, it is probably easiest to use Pythons built in list operations, but you can use loops if you so desire. Once you have finished, compare your answer to the one supplied below. 

In [6]:
## Write your code in here
def is_palindrome(pal):
    ???
test_is_palindrome()


Object `` not found.


AssertionError: 

In [9]:
def is_palindrome(pal):
    return pal[::-1]==pal
test_is_palindrome()


Once again our code is passing all our tests. It may feel as though we have finished with our `is_palindrome`
function, it seems to be working ok. But what about some more "real world" cases. For example, which happens if there is leading or trailing whitespace in the string we are checking?

In [8]:
def test_is_palindrome():
    assert is_palindrome("a") == True
    assert is_palindrome("bb")==True
    assert is_palindrome("  kayak ")==True
test_is_palindrome()

AssertionError: 

Our code didn't pass our new test. So we need to take out the whitespace at the start and end of the string "  kayak ". Python strings have inbuilt methods which can do this for us. I suggest you try google your way out of this one. In all seriousness, programming involves a fair amount of googling; don't try and re-invent the wheel but at the same time don't blindy copy and paste code. [Stackoverflow](Stackoverflow.com) is an excellent resource for answering programming related questions; there is a good chance someone has already asked the question you have.

In [11]:
## Write your code in here
def is_palindrome(pal):
    ???
test_is_palindrome()

Object `` not found.


AssertionError: 

In [12]:
## Completed answer
def is_palindrome(pal):
    pal=pal.strip()
    return pal[::-1]==pal

test_is_palindrome()


Now that we can handle whitespace, what about other characters, such as numbers and capital letters? 

In [13]:
def test_is_palindrome():
    assert is_palindrome("a") == True
    assert is_palindrome("bb")==True
    assert is_palindrome("  kayak")==True
    assert is_palindrome("d33d")==True
    assert is_palindrome(" Noon   ")==True
test_is_palindrome()

AssertionError: 

In [10]:
## Write your code in here
def is_palindrome(pal):
    ???
test_is_palindrome()

Object `` not found.


AssertionError: 

In [6]:
## Completed answer
def is_palindrome(pal):
    pal=pal.lower()
    pal=pal.strip()
    return pal[::-1]==pal

test_is_palindrome()

AssertionError: 

Finally, what about punctuation? should we specify that it is part of the palindrome, or ignore it? If you are feeling up for it, try implement a version of 
    is_palindrome
that removes all punctuation from the test string. Check that it passes the new test added.

In [11]:
def test_is_palindrome():
    assert is_palindrome("a") == True
    assert is_palindrome("bb")==True
    assert is_palindrome("ab3d")==False
    assert is_palindrome("  kayak")==True
    assert is_palindrome(" Noon   ")==True
    assert is_palindrome("\"Sirrah! Deliver deified desserts detartrated!\" stressed deified, reviled Harris.\"")==True
test_is_palindrome()




a
bb
ab3d
kayak
noon
"sirrah! deliver deified desserts detartrated!" stressed deified, reviled harris."


AssertionError: 

In [5]:
## Write your code in here
def is_palindrome(pal):
    ???
test_is_palindrome()

Object `` not found.


AssertionError: 

In [15]:
import re

def is_palindrome(pal):
    pal=pal.lower()
    pal=pal.strip()
    pal=re.sub(r"[^a-zA-Z0-9\s]",'',pal)
    print pal
    
    return pal[::-1]==pal    
test_is_palindrome()

a
bb
ab3d
kayak
noon
sirrah deliver deified desserts detartrated stressed deified reviled harris


So our palindrome function is now pretty comprehensive. At this point you maybe be thinking, we are now actually done. But so far we have only considered the standard english character set. Should we consider other writing systems or the inclusions of symbols ↟?

## Naming standards for our tests

So far we have been lumping all our tests together inside a single function. There are a couple of issues with this. When a test fails, the remaining tests in the function  will not run. Secondly, its hard to know exactly which test failed and why, we have to inspect the arguments we passed to our palindrome function to work out what we we were testing. So the first thing we need to do is split our tests into different functions, one function per test. After that we need to name our functions so that we can work out what they are testing solely based on the name. 

So how do we name our test functions? There are no hard and fast rules, but a common way to name a test is **test\_\[test\_name\]**. Following these conventions our palindrome tests would look like the following

In [14]:

def test_single_character_palindrome():
    assert is_palindrome("a") == True
    
def test_two_same_character_palindrome():
    assert is_palindrome("bb")==True
    
def test_numbers_not_palindrome():
    assert is_palindrome("ab3d")==False
    
def test_spacing_palindrome():
    assert is_palindrome("  kayak")==True
    
def test_capitals_palindrome():
    assert is_palindrome(" Noon   ")==True
    
def test_punctuation_palindrome():
    assert is_palindrome("\"Sirrah! Deliver deified desserts detartrated!\" stressed deified, reviled Harris.")==True

When renaming the tests, you can also include the expected state of the test, as shown above where we have indicated whether or not we expect it to be a palindrome. We can extend our naming convention to **test\_\[test\_name\]\_[test\_outcome]** if we think its helpful. 

## Organising Tests

While you can include tests inside the python modules you write, it is better to put them in a external module. When you are making changes to your code, it will be more straightforward if they are elsewhere. Having fewer lines of code makes life easier  when renaming and moving code around when refactoring. Seperate tests by functionality into different modules. With our palindrome example, if we also had a function which **[DOES SOMETHING ELSE]** we would have one module to test each functionality. 

# Testing with Pytest

At this point, you may be wondering that wouldn't it be very time consuming and annoying to run each test function we have written one by one? Software developers being the lazy people they are have use software that automate the testing process. There are several popular testing packages for Python. In this exercise we are going to use the Pytest library. Pytest is very powerful and has a wide number of features, but is easy to get started with. As you become more proficent with Python, you will find more and more of its features useful. Pytest lets us write tests similar to how we have been doing so in this notebook, writing functions that use assert to compare the output of the tested function against the expected return value. Pytest also sniffs out python test files, any files that start with "test\_" or finish with "\_test" (excluding the .py file extention) will be found and run by Pytest by default.

## Basic Usage

Using pytest is quite straightforward. Lets continue with our palindrome example. In our palindrome folder we have the following files and folders:

<img src="folder.png",width=500,height=500>

In the **tests** folder we have another another testing file called `test_2_palindrome.py`

As mentioned before pytest will sniff out any test files with the right prefix or suffix. If we run `pytest` from the terminal in the `tests` folder it will run both `palindrome_1_test.py` and `test_2_palindrome.py`. The default behaviour of pytest is to sniff out any tests with the naming convention mentioned above in the folder it was called from and all subfolders. It is possible to run pytest from within the test folder, but setting it up to do so is more hassle than it is worth.

After running pytest we will get the following output


Pytest is telling use which of our tests failed. Since we updated our naming conventions, it is now more clear on about how our code is failing to pass our tests. If we keep a terminal open we can easily move back and forth from running tests to writing code as we follow the TDD cycle. 

If you would rather read a html file, you can install a plugin [pytest-html](https://pypi.python.org/pypi/pytest-html), which will create html test reports for you, which may be a bit easier on the eye. 

# Final Note

Scientific software can be a difficult domain to create tests for. Often you are trying to compute the answer to the question you don't know the answer to a priori. Ensuring your code is well tested helps ensure a novel result is not the outcome of a bug, but at worst an error in the computational method or algorithm you have devised, you have might have the wrong design but it is implemented correctly. TDD will help narrow down the domain where you error lies. 